In [1]:
import numpy as np
import pandas as pd

## Loading the data from separate files to combine in one 

In [2]:
# Load CSV files into individual Pandas DataFrames, setting `low_memory=False` to optimize memory usage 
data_ab = pd.read_csv('/kaggle/input/canada-housing/data_ab.csv', low_memory=False)
data_bc = pd.read_csv('/kaggle/input/canada-housing/data_bc.csv', low_memory=False)
data_mb = pd.read_csv('/kaggle/input/canada-housing/data_mb.csv', low_memory=False)
data_nb = pd.read_csv('/kaggle/input/canada-housing/data_nb.csv', low_memory=False)
data_nl = pd.read_csv('/kaggle/input/canada-housing/data_nl.csv', low_memory=False)
data_ns = pd.read_csv('/kaggle/input/canada-housing/data_ns.csv', low_memory=False)
data_nt = pd.read_csv('/kaggle/input/canada-housing/data_nt.csv', low_memory=False)
data_on = pd.read_csv('/kaggle/input/canada-housing/data_on.csv', low_memory=False)
data_pe = pd.read_csv('/kaggle/input/canada-housing/data_pe.csv', low_memory=False)
data_sk = pd.read_csv('/kaggle/input/canada-housing/data_sk.csv', low_memory=False)
data_yt = pd.read_csv('/kaggle/input/canada-housing/data_yt.csv', low_memory=False)

In [3]:
# Concatenate all DataFrames along the rows (`axis=0`) to create a unified dataset

df = pd.concat([data_ab, data_bc, data_mb, data_nb, data_nl, 
                data_ns, data_nt, data_on, data_pe, data_sk,
                data_yt], axis=0) 
df.shape

(133725, 396)

The dataset is quite big. Let's drop null rows and columms

In [4]:
# Drop duplicated rows
df = df.drop_duplicates()

# Drop duplicate columns
df = df.T.drop_duplicates().T
df.shape

(98649, 32)

With option pd.set_option('display.max_columns', None) we can see all features

In [5]:
pd.set_option('display.max_columns', None)
df.head()

,streetAddress,addressLocality,addressRegion,postalCode,latitude,longitude,description,price,priceCurrency,property-beds,property-baths,property-sqft,Acreage,Air Conditioning,Basement,Bath,Exterior,Features,Fireplace,Garage,Heating,MLS® #,Property Tax,Property Type,Roof,Sewer,Square Footage,Subdivision,Waterfront,Parking,Flooring,Fireplace Features
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"1701 Coursier Avenue Unit# 1301, Revelstoke, B...",Revelstoke,BC,V0E2S3,50.976585,-118.173149,"Don't miss out on this rare 3 bedroom, 2 bath ...",839000.0,CAD,3.0,2.0,891,NaN,NaN,NaN,0.0,NaN,"Underground, Heated Garage",NaN,NaN,Heat Pump,10334677,"$3,478",Condo,NaN,Municipal sewage system,891 SQFT,NaN,NaN,"Underground, Heated Garage","Tile, Carpeted, Vinyl",NaN


There is no need in data without address or price, so we can drop all rows with null values in "streetAddress", "addressLocality", "addressRegion", "price" columns:

In [6]:
df = df.dropna(subset=["streetAddress", "addressLocality", "addressRegion", "price"])

In [7]:
df.shape

(97811, 32)

# Square Footage and property-sqft

Columns 'Square Footage', 'property-sqft' have the same information. Let's combine them in one column. 

In [8]:
df[['Square Footage', 'property-sqft']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 97811 entries, 7 to 211
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Square Footage  46732 non-null  object
 1   property-sqft   46489 non-null  object
dtypes: object(2)
memory usage: 2.2+ MB


In [9]:
df.loc[:, 'Square Footage new'] = df['Square Footage'].str.replace(' SQFT', '', regex=False)
df.loc[:, 'Square Footage new'] = df['Square Footage new'].fillna(df['property-sqft'])

df[['property-sqft', 'Square Footage', 'Square Footage new']].head()

,property-sqft,Square Footage,Square Footage new
7,891,891 SQFT,891
8,"1,881","1,881 SQFT","1,881"
9,912,912 SQFT,912
10,"4,374","4,374 SQFT","4,374"
11,NaN,NaN,NaN


In [10]:
df[['Square Footage new']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 97811 entries, 7 to 211
Data columns (total 1 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Square Footage new  46732 non-null  object
dtypes: object(1)
memory usage: 1.5+ MB


# Acreage

In [11]:
df.Acreage.head()

7      NaN
8     0.32
9      NaN
10    0.69
11    0.14
Name: Acreage, dtype: object

"Acreage" typically refers to a large piece of land, usually measured in acres. In real estate, it implies that the property includes a significant amount of land, often used for farming, ranching, or simply as a large private estate. So, Nan values can be replaced to 0

In [12]:
df["Acreage"] = df["Acreage"].fillna(0)
df.Acreage.head()

<ipython-input-12-9931aeb8a7af>:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["Acreage"] = df["Acreage"].fillna(0)


7     0.00
8     0.32
9     0.00
10    0.69
11    0.14
Name: Acreage, dtype: float64

# property-bath

In [13]:
df[['property-baths', 'Bath']].describe()

,property-baths,Bath
count,97041.0,26451.0
unique,26.0,9.0
top,2.0,1.0
freq,28083.0,17532.0


In [14]:
# Get unique values from both columns
unique_values = pd.Index(df['property-baths'].dropna().unique()).union(df['Bath'].dropna().unique())

# Create a DataFrame with unique values as index
counts_df = pd.DataFrame(index=unique_values, columns=['property-baths', 'Bath'])

# Count occurrences and fill the DataFrame
counts_df['property-baths'] = df['property-baths'].value_counts().reindex(unique_values, fill_value=0)
counts_df['Bath'] = df['Bath'].value_counts().reindex(unique_values, fill_value=0)

print(counts_df)

       property-baths   Bath
0.0             14428   7630
1.0             16867  17532
2.0             28083   1212
3.0             20532     59
4.0             11252     14
5.0              3020      1
6.0              1469      0
7.0               654      0
8.0               359      0
9.0               186      1
10.0               88      0
11.0               45      0
12.0               31      0
13.0                7      0
14.0                4      0
15.0                3      0
16.0                1      0
17.0                1      0
18.0                2      0
19.0                1      0
21.0                2      1
23.0                1      0
25.0                1      0
26.0                2      0
42.0                1      0
802.0               0      1
803.0               1      0


In [15]:
pd.set_option('display.max_colwidth', None)

df[df['property-baths'] == 42.0][['property-baths', 'Bath', 'description', 'price']]

,property-baths,Bath,description,price
846,42.0,NaN,"Discover this remarkable 24,000 sq ft commercial-residential mixed-use property offering incredible income potential and a proven high return on investment. Situated in a prime location, this asset is a standout opportunity for seasoned investors and developers alike.Key Features:1.Outstanding Income Return:oCurrent net profit: $675,557.13 annually, equating to a 12.75% net income return.oPost-conversion potential: $766,182 annual net profit, yielding an enhanced 14.45% net return.2.High-Value Asset with Significant Upside:oAppraised at $7.25M by a bank-approved appraiser.oMarket value projected to exceed $9M upon conversion to a 100% residential property.3.Existing Configuration:o12,000 sq ft of prime retail and office space.oResidential units: 4 studios, 8 one-bedroom apartments, and 8 two-bedroom apartments.4.Conversion Potential:oAfter transformation, the property will feature 6 studios, 8 one-bedroom apartments, and 28 two-bedroom apartments, catering to high-demand residential needs.5.Renovation Incentives:oGuaranteed renovation cost: Only $650,000, after factoring in city-supported CIP grants and interest-free loans.oBoosted rental income: Conversion results in a $90,625 annual increase in net rental income.6.Favorable Financing Options:oCMHC loan eligibility upon issuance of building permits.oLoan covers 100% of conversion costs at a competitive 4% interest rate.Why This Property?With strong current returns, significant upside from planned conversions, and the city-backed incentives to minimize costs, this property represents a rare blend of high yields, manageable risk, and long-term value appreciation.Don't miss this exceptional investment opportunity in the thriving City of Sudbury. (id:38686)",5388888.0


In [16]:
df[df['property-baths'] == 803][['property-baths', 'Bath', 'description', 'price']]

,property-baths,Bath,description,price
9364,803.0,802.0,"This 2 bedroom home is located on Blackhead Road and looking for new owners. The main floor has an eat in kitchen, 2 bedrooms, living room, 4 piece bathroom and a porch. There is a door located off the kitchen to a large patio that would be great for those summer BBQ's. There is a room developed in the basement that would make a great office or a third bedroom for the teenager who need or want their own space. There is also a large rooms that would make an ideal rec room when it is completed. Lots of room in the back garden and an area that is fenced and great for the animals to run and play. There is also a pad already poured if someone is looking to build an outbuilding. This home required some work like painting, flooring and any upgrades you may want to do but there is no doubt that this home has lots of potential. (id:38686)",198000.0


In [17]:
df.loc[df['property-baths'] == 803, 'property-baths'] = 1

In [18]:
df[['property-baths']].value_counts()

property-baths
2.0               28083
3.0               20532
1.0               16868
0.0               14428
4.0               11252
5.0                3020
6.0                1469
7.0                 654
8.0                 359
9.0                 186
10.0                 88
11.0                 45
12.0                 31
13.0                  7
14.0                  4
15.0                  3
18.0                  2
21.0                  2
26.0                  2
16.0                  1
17.0                  1
19.0                  1
23.0                  1
25.0                  1
42.0                  1
Name: count, dtype: int64

# Features

In [19]:
df.Features.head()

7                          Underground, Heated Garage
8     Attached Garage, RV, See Remarks, Heated Garage
9                                            Aluminum
10                                    Attached Garage
11                                          Other, RV
Name: Features, dtype: object

The column 'Features' contains detailed information about a house, including parking, garage, pool, heating, garden, and other. Some rows may have multiple features listed, while others might only contain a few or even none. Additionally, the 'Garage' column can sometimes be empty, even when garage-related details are present in the 'Features' column. This makes it crucial to extract and analyze all values from 'Features' to ensure no important information is missed.

In [20]:
df['Features'].unique().shape

(5338,)

We have 5,338 different values in the 'Feature' column. The problem is that each cell can contain a list of information, with potential relationships between features in the dataset. We can define some keywords for each feature, extract relevant values, and fill in the gaps.

In [21]:
##### Extract unique features
def get_unique_values(column):
    
    unique_features = set()
    
    for features in df[column].dropna():
        # Remove brackets and extra quotes
        cleaned_features = features.replace("[", "").replace("]", "").replace('"', '').replace("'", "")
        # Split by comma
        feature_list = [f.strip() for f in cleaned_features.split(",")]
        unique_features.update(feature_list)

    return unique_features

In [22]:
unique_features = get_unique_values('Features')
print(len(unique_features), unique_features)

268 {'', 'Garage; Single', 'Alley Access', 'Carport; Multiple', 'Carport & Garage', 'Stream/Pond', 'Park/Reserve', 'Public Swimming Pool', 'Golf Nearby', 'Private Electric Vehicle Charging Station(s)', 'Heated Driveway', 'Flat Site', 'Garage Underbuilding', 'Rear', 'Interlocked', 'Concrete', 'Playground Nearby', 'Garden', 'Electric', 'Double Carport', 'Attached Garage', 'Garage Double', 'Parking Lot', 'Aggregate', 'Carport Quad+', 'Concrete/Stucco', 'Corner Lot', 'Aluminum/Vinyl', 'Heated Garage', '2 Car Attached', 'Fenced', 'Public Electric Vehicle Charging Station(s)', 'Cul-De-Sac', 'Colour Loc', 'Varies by Unit', 'Gated', 'On Site', 'Conventional', 'Rock Imitation', 'Asbestos', 'Quad or More Detached', 'Double Garage Attached', 'Multiple Driveways', 'Inside Entry', 'Single Carport', 'Carport; Single', 'Parkade', 'Interlocking Driveway', 'Attached', 'River View', 'Garage Quad+', 'Shared', 'Breezeway', 'Visitor Parking', 'Stall', 'Garage Door Opener', 'None', 'Free Standing Metal', '2

Now we got 268 different unique values. Let's fill the gap in Garage column

## Garage

In [23]:
df.Garage.value_counts()

Garage
Yes    9602
Name: count, dtype: int64

Column 'Garage' contains only 'Yes' or 'No' values. The information about type of garage, how namy of them in the house could contain columns 'Feature' or 'Parking'. Let's combine them.

In [24]:
unique_parking_features = get_unique_values('Parking') | unique_features
print(len(unique_parking_features), unique_parking_features)

268 {'', 'Garage; Single', 'Carport & Garage', 'Park/Reserve', 'Public Swimming Pool', 'Heated Driveway', 'Flat Site', 'Rear', 'Double Carport', 'Carport Quad+', 'Parking Lot', 'Heated Garage', 'Public Electric Vehicle Charging Station(s)', 'Conventional', 'Quad or More Detached', 'Double Garage Attached', 'Multiple Driveways', 'Interlocking Driveway', 'Attached', 'River View', 'Shared', 'Visitor Parking', 'Stall', '2 Car Detached', 'Parking Pad', 'Carport Triple', '6 Car Detached', 'Wood siding', 'Private Setting', 'Gravel', 'Underground Parking', 'Open', 'RV Parking Avail.', 'Steel', 'Playground', 'Covered', 'Level Land', 'Brick Imitation', 'Cedar', 'Picnic Area', 'Wood Siding', 'Paved Lane', 'Stove', 'Asphalt', 'Brick', 'Ski Hill Nearby', 'Low Maintenance Landscape', '220 Volt Wiring', '2', 'Landscaped', 'Direct vent', 'Partially Landscaped', 'Subdividable Lot', 'Wood shingles', 'Lake Access Property', '4 Car Attached', 'Gas', 'Roughed in', 'Private Fishing', 'Block Driveway', 'Gara

Let's select from this list only garage related terminology:

In [25]:
garage_keywords = {"garage", "carport", "grge", "attached", "detached", "tandem", "car"}

# The selected keywords cover different terms related to garages and carports:

#     "garage" → Captures any mention of a garage.
#     "carport" → Includes carports, which serve a similar function.
#     "grge" → A common abbreviation for "garage" (e.g., "DetachedGrge/Carport").
#     "attached" → Covers garages that are attached to the house.
#     "detached" → Covers garages that are separate from the house.
#     "tandem" → Represents tandem garages, where cars are parked one behind the other.
#     "car" → Captures features mentioning a car (e.g., "2 Car Attached", "3 Car Detached").

garage_list = []


for item in unique_parking_features:
    lower_item = item.lower()
    if any(keyword in lower_item for keyword in garage_keywords):
        garage_list.append(item.lower())

garage_list.remove('no garage')

print("Garage-related items:", garage_list)

Garage-related items: ['garage; single', 'carport & garage', 'double carport', 'carport quad+', 'heated garage', 'quad or more detached', 'double garage attached', 'attached', '2 car detached', 'carport triple', '6 car detached', '4 car attached', 'garage; double', 'rv garage', 'in garage electric vehicle charging station(s)', 'triple garage attached', 'tandem parking', 'garage faces side', '4 car detached', 'carport', 'garage; underground', 'garage triple', '6 car attached', '3 car detached', '1 car detached', 'detached garage', 'single garage detached', '1 car attached', 'detachedgrge/carport', 'carport; multiple', 'garage underbuilding', 'attached garage', 'garage double', '2 car attached', 'single carport', 'carport; single', 'garage quad+', 'garage door opener', 'grge/double tandem', '3 car attached', '5 car attached', 'detached', 'workshop in garage', 'tandem', 'double garage detached', 'quad or more attached', 'garage faces rear', 'single garage attached', 'attached carport', 'g

In [26]:
def check_keywords(features, keywords):

    try:
        cleaned_features = features.replace("[", "").replace("]", "").replace('"', '').replace("'", "").replace(";", ",")
        list_to_check = [f.strip().lower() for f in cleaned_features.split(',')]
        common = set(list_to_check) & set(keywords)
        
        if common: 
            return 'Yes'  
            
    except AttributeError:
        return 'No'
    
    return 'No'

The function 'check_keywords' will check if column 'Features' and 'Parking' contain at least one pfrase from the list 'garage_list'. In this case, the nan value in column 'Garage' will be changes to "Yes".

In [27]:
df['Garage new'] = df['Garage']
df.loc[df.Garage.isna(), 'Garage new'] = df.loc[df.Garage.isna(), 'Features'].apply(lambda x: check_keywords(x, garage_list))
df['Garage new'].value_counts()

Garage new
Yes    55223
No     42588
Name: count, dtype: int64

Some rows can have information about garage in Parking column

In [28]:
df.Parking.unique()

array(['Underground, Heated Garage',
       'Attached Garage, RV, See Remarks, Heated Garage', nan, ...,
       '["No Garage", "Covered"]',
       '["1 Car Detached", "Garage/Laneway Suite"]',
       '["2 Car Attached", "Parking Pad", "RV Parking"]'], dtype=object)

In [29]:
df.loc[df.Garage == 'No', 'Garage new'] = df.loc[df.Garage == 'No', 'Parking'].apply(lambda x: check_keywords(x, garage_list))
df['Garage new'].value_counts()

Garage new
Yes    55223
No     42588
Name: count, dtype: int64

The number of not-null values does not changed. So all possible values were filled from 'Features' column.

## Let's do the same for column 'Parking'

In [30]:
df.Parking.value_counts()

Parking
[]                                                 11111
Attached Garage                                     5560
Underground                                         4924
["Attached Garage"]                                 3793
Garage                                              2566
                                                   ...  
Driveway, Open, RV Access/Parking                      1
Additional Parking, Carport                            1
Additional Parking, Carport, Driveway, Open            1
Carport; Multiple, DetachedGrge/Carport, Open          1
["2 Car Attached", "Parking Pad", "RV Parking"]        1
Name: count, Length: 4907, dtype: int64

In [31]:
parking_list = ["2 Outdoor Stalls", "Add. Parking Avail.", "Additional Parking", "Assigned",
                "Carport", "Carport & Garage", "Carport Double", "Carport Quad+", "Carport Triple", 
                "Carport; Multiple", "Carport; Single", "Multiple Driveways", "Parkade", "Parking Lot", 
                "Parking Pad", "Parking Pad Cement/Paved", "Parking Space(s)", "Parking Spaces", 
                "RV", "RV Access/Parking", "RV Gated", "RV Hookup", "RV Parking", "RV Parking Avail.", 
                "Shared Driveway", "Stall", "Tandem Parking", "Underground", "Underground Parking", 
                "Visitor Parking"
                ]

parking_list = [f.lower() for f in parking_list]

We'll replace values in a Parking column to 'Yes' and "No'

In [32]:
# first, copy the column
df['Parking new'] = df['Parking']

# if the column 'Parking' has one of the words from the parking_list, it will be replaced to 'Yes'
df['Parking new'] = df['Parking new'].apply(lambda x: check_keywords(x, parking_list))

# if the column 'Features' has one of the words from the parking_list, it will be replaced to 'Yes' in column 'Parking'
df.loc[df['Parking new'] == 'No', 'Parking new'] = df.loc[df['Parking new']  == 'No', 'Features'].apply(lambda x: check_keywords(x, parking_list))

df['Parking new'].value_counts()

Parking new
No     67834
Yes    29977
Name: count, dtype: int64

## Basement

In [33]:
df.Basement.value_counts()

Basement
[]                                                      5502
["Finished", "Full"]                                    3293
Finished, Full                                          3194
Finished, N/A                                           2178
["Unfinished", "Full"]                                  1481
                                                        ... 
["Crawl Space", "Finished", "Walk-Out Access"]             1
Not Full Height, Partially Finished, Walk-Out Access       1
["Partial", "Unfinished", "Walk-Out Access"]               1
["With Windows"]                                           1
["Cellar", "Partial Basement", "Unfinished"]               1
Name: count, Length: 636, dtype: int64

Let's extract unique features and select related to 'Basement' values

In [34]:
unique_basement_features = get_unique_values('Basement')
print(len(unique_basement_features), unique_basement_features)

38 {'Partial Basement', 'Slab', '', 'Full', 'Remodeled Basement', 'Separate/Exterior Entry', 'Fully Finished', 'Crawl space', 'Remodeled', 'No Basement', 'Finished', 'See Remarks', 'Walk-Out Access', 'Walk-Up To Grade', 'Other', 'Dugout', 'Unfinished', 'With Windows', 'Separate entrance', 'Walk out', 'Walkout', 'Crawl', 'Partially finished', 'Full Basement', 'N/A', 'Not Full Height', 'Not applicable', 'Apartment in basement', 'Crawl Space', 'Walk-up', 'Suite', 'Walk-Out To Grade', 'Partially Finished', 'Cellar', 'Not Applicable', 'Partial', 'None', 'Unknown'}


#### Finished (These basements or spaces are completed or usable, often with features like exterior access or fully finished rooms.)
- "Full Basement": A basement that spans the entire footprint of the house and is fully developed for use.
- "Full": Another way to refer to a completely finished basement.
- "Dugout": A basement or below-ground space that has been excavated (dug out) to provide additional space.
- "Fully Finished": A basement that is 100% completed with flooring, walls, ceiling, and sometimes utilities.
- "Finished": A general term for a basement that is move-in ready.
- "Remodeled Basement": A basement that was previously unfinished or outdated but has been renovated.
- "Apartment in Basement": A basement that has been converted into a separate living unit with amenities like a kitchen and bathroom.
- "Suite": A finished basement space designed as a separate living area, often with its own entrance.
- "Walk-Out Access": A basement with a door leading directly outside, typically to ground level.
- "Walk-Out To Grade": Similar to "Walk-Out Access," meaning the basement exits directly to ground level.
- "Walk-Up To Grade": A basement that has a stairway leading up to ground level for external access.
- "Walk-Out / Walkout": Another way to describe a basement that has an external door leading outside.
- "Walk-Up": A basement with stairs leading to an outside entrance.
- "With Windows": A basement that has windows, often allowing natural light and making it more livable.
- "Separate/Exterior Entry": A basement that has a private entrance from outside, making it ideal for rental units or guest spaces.
- "Separate Entrance": Similar to "Separate/Exterior Entry," meaning the basement has independent access from outside.

#### Partial (These basements or spaces are partially finished or incomplete.)
- "Partially Finished" / "Partially finished": Some areas are finished, but others may be unfinished or under construction.
- "Partial" / "Partial Basement": The basement is smaller than the home's full footprint or has limited finished space.
- "Not Full Height": A basement that does not have standard ceiling height, making it less functional as a full living space.
- "Cellar": An older term, often referring to basements used for storage, with lower ceilings and minimal finishing.

#### No Basement (These properties either don't have basements or have limited, unusable spaces underneath.)
- "Unfinished": A basement that exists but has no completed flooring, walls, or ceiling.
- "No Basement": The property does not include a basement.
- "Slab": The house is built directly on a concrete slab, with no basement underneath.
- "Crawl Space" / "Crawl": A shallow space under the house, not intended for living but used for ventilation or storage.
- "None": No basement or similar structure exists in the property.
- "N/A": Not available or not provided.
- "Not Applicable": The basement condition does not apply, possibly because there is no basement or it's an irrelevant listing.

In [35]:
basement_types = {'Finished': ['full basement', 'full', 'dugout', 'fully finished', 'finished', 'remodeled basement', 
                               'apartment in basement', 'suite', 'walk-out access', 'walk-out to grade', 'walk-up to grade', 
                               'walk-out', 'walkout', 'walk-up', 'with windows', 'separate/exterior entry', 'separate entrance'], 
                  'Partial': ['partially finished', 'partial', 'partially finished', 'partial basement', 
                              'not full height', 'cellar'], 
                  'No basement': ['unfinished', 'no basement', 'slab', 'crawl space', 'crawl', 'none', 
                                  'not applicable', 'n/a']}

In [36]:
def check_type(features, types):

    try:
        cleaned_features = features.replace("[", "").replace("]", "").replace('"', '').replace("'", "").replace(";", ",")
        list_to_check = [f.strip().lower() for f in cleaned_features.split(',')]

        for basement_type, basement_type_list in types.items():
            if set(list_to_check) & set(basement_type_list): 
                return basement_type
        return np.nan

    except AttributeError:
        return np.nan

In [37]:
df['Basement new'] = df['Basement']
df['Basement new'] = df['Basement new'].apply(lambda x: check_type(x, basement_types))
df['Basement new'].value_counts()

Basement new
Finished       30106
No basement     5471
Partial         2055
Name: count, dtype: int64

## Exterior

In [38]:
df.Exterior.value_counts().head()

Exterior
[]           9829
Brick        7340
["Brick"]    3220
["Vinyl"]    3017
Concrete     2920
Name: count, dtype: int64

In [39]:
unique_exterior_features = get_unique_values('Exterior')   
print(len(unique_exterior_features), unique_exterior_features)

140 {'', 'Stream/Pond', 'Lighting', 'Boat Slip', 'Park/Reserve', 'Public Swimming Pool', 'Rock imitation', 'Golf Nearby', 'Water Feature', 'Flat Site', 'Concrete', 'Playground Nearby', 'Garden', 'Concrete/Stucco', 'Security System', 'Corner Lot', 'Aluminum/Vinyl', 'Colour Loc', 'Fenced', 'Cul-De-Sac', 'Rock Imitation', 'Asbestos', 'River View', 'Electric Grill', 'None', 'Outdoor Shower', 'Fencing: Partial', 'View Downtown', 'Wood siding', 'Balcony/Patio', 'Private Setting', 'Aluminum siding', 'Gated Community', 'RV Hookup', 'Steel', 'Shopping Nearby', 'Playground', 'Shingles', 'BBQ gas line', 'Level Land', 'Brick Imitation', 'View City', 'Sloping Lot', 'Cedar', 'Picnic Area', 'Wood Siding', 'Paved Lane', 'Gas Grill', 'Permeable Paving', 'Concrete Block', 'Concrete Siding', 'Asphalt', 'Brick', 'Low Maintenance Landscape', 'Ski Hill Nearby', 'Schools', 'Landscaped', 'Private Yard', 'Covered Courtyard', 'Built-in Barbecue', 'Partially Landscaped', 'Subdividable Lot', 'Dock', 'Lake Access 

In [40]:
exterior_materials = {
    'Metal': ['aluminum', 'aluminum siding', 'aluminum/vinyl', 'colour loc', 'metal', 'steel'],
    'Brick': ['brick', 'brick facing', 'brick imitation', 'brick veneer'],
    'Concrete': ['concrete', 'concrete siding', 'concrete/stucco', 'concrete block', 'insul brick', 'stucco'],
    'Wood': ['cedar', 'cedar shingles', 'cedar siding', 'wood', 'wood siding', 'wood shingles', 'wood shingles', 'wood siding'],
    'Composite': ['composite siding', 'hardboard', 'masonite', 'shingles'],
    'Vinyl': ['vinyl', 'vinyl siding', 'asbestos', 'siding']
}

In [41]:
df['Exterior new'] = df['Exterior']
df['Exterior new'] = df['Exterior new'].apply(lambda x: check_type(x, exterior_materials))
df['Exterior new'].value_counts()

Exterior new
Brick        20344
Vinyl        10806
Concrete      7639
Metal         3561
Wood          2940
Composite      784
Name: count, dtype: int64

## Fireplace

In [42]:
print(df.Fireplace.shape[0]) 
df.Fireplace.value_counts()[:10]

97811


Fireplace
[]         21932
Yes        19967
1           3291
["1"]       2717
0           1454
["2"]       1256
["0"]       1171
2           1102
Gas         1014
["Gas"]      894
Name: count, dtype: int64

Column 'Fireplace' contain information in different appearence. Let's reduce it ot 'Yes' and 'No'. Also we need to get information from the column 'Fireplace Features'.

In [43]:
df['Fireplace new'] = df['Fireplace']

df.loc[df['Fireplace new'].isin(['0', '[]','["0"]']), 'Fireplace new'] = np.nan
df.loc[df['Fireplace new'].notna(), 'Fireplace new'] = 'Yes'
df.loc[~df['Fireplace new'].notna(), 'Fireplace new'] = 'No'


In [44]:
df['Fireplace new'].value_counts()

Fireplace new
No     63356
Yes    34455
Name: count, dtype: int64

# Heating

In [45]:
df.Heating.unique().shape

(2642,)

In [46]:
unique_heating_features = get_unique_values('Heating')
print(len(unique_heating_features), unique_heating_features)

85 {'', 'Fireplace(s)', 'Outside Furnace', 'Steam', 'Air Conditioning', 'Not known', 'Radiant Floor', 'Separate Water Heaters', 'Electric', 'Floor heat', 'Fireplace Insert', 'Zoned', 'Forced air', 'Forced Air-2', 'Furnace', 'Ceiling', 'Radiator', 'Coal Stove', 'None', 'Geo Thermal', 'See remarks', 'No heat', 'Natural Gas', 'Pellet Stove', 'Coal', 'Radiant Heat', 'Wall Mounted Heat Pump', 'Make-up Air', 'In Floor', 'High Efficiency', 'Overhead Heaters', 'Stove', 'Baseboard', 'Ductless', 'Boiler', 'Wall Furnace', 'Radiant/Infra-red Heat', 'Forced Air-1', 'Water', 'Radiant', 'Gravity', 'In Floor Heating', 'Oil', 'In Floor Heat System', 'Humidity Control', 'Forced Air', 'Other', 'Central heating', 'See Remarks', 'Mixed', 'Floor Furnace', 'Geothermal', 'Exhaust Fan', 'Baseboard heaters', 'Propane', 'Mid  Efficiency', 'Heat Recovery Ventilation (HRV)', 'Gravity Heat System', 'Hot water radiator heat', 'Pellet', 'Propane Gas', 'Solar', 'Space Heater', 'Central', 'Heat Pump', 'Sep. HVAC Units'

Let's divide them in categories:

In [47]:
heating_categories = {
    'forced air': ['forced air', 'forced air-1', 'forced air-2', 'furnace'],
    'boiler': ['boiler', 'hot water', 'hot water radiator heat', 'steam', 'steam radiator'],
    'heat pump': ['central heat pump', 'heat pump', 'wall mounted heat pump'],
    'radiant': ['radiant', 'radiant heat', 'radiant ceiling', 'radiant floor', 
                'radiant/infra-red heat', 'baseboard', 'baseboard heaters', 'electric baseboard units'],
    'fireplace': ['fireplace(s)', 'fireplace insert', 'wood stove', 'pellet stove', 'coal stove', 'stove'],
    'space heat': ['space heater', 'space heaters', 'wall furnace', 'floor furnace', 
                   'floor model furnace', 'overhead heaters', 'overhead unit heater', 'ductless'],
    'alt heat': ['geo thermal', 'geothermal', 'solar', 'gravity', 'gravity heat system', 
                 'oil', 'propane', 'propane gas', 'coal'],
    'no heat': ['no heat', 'none'],  # only contains no heat options
    'other': ['mixed', 'combination', 'sep. hvac units'],  # these suggest non-specific or mixed systems
}

In [48]:
df['Heating new'] = df['Heating']
df['Heating new'] = df['Heating new'].apply(lambda x: check_type(x, heating_categories))

df['Heating new'] .value_counts()

Heating new
forced air    53347
radiant       12053
heat pump      5606
boiler         3924
alt heat        529
fireplace       238
no heat          76
space heat       74
other            64
Name: count, dtype: int64

# Flooring

In [49]:
df.Flooring.value_counts()

Flooring
[]                                                                   21368
Laminate                                                              2292
Hardwood                                                              1905
["Laminate"]                                                          1225
["Hardwood"]                                                           719
                                                                     ...  
["Hardwood", "Laminate", "Carpeted", "Other", "Wood"]                    1
["Hardwood", "Laminate", "Carpeted", "Linoleum", "Vinyl", "Cork"]        1
["Hardwood", "Laminate", "Ceramic Tile", "Cork"]                         1
["Tile", "Carpeted", "Ceramic", "Linoleum", "Wood"]                      1
["Laminate", "Tile", "Vinyl", "Wood"]                                    1
Name: count, Length: 2105, dtype: int64

In [50]:
unique_flooring_features = get_unique_values('Flooring')
print(len(unique_flooring_features), unique_flooring_features)

42 {'', 'Carpet Over Softwood', 'Carpeted', 'Granite', 'Porcelain Tile', 'Softwood', 'Cushion/Lino/Vinyl', 'Non-Ceramic Tile', 'Concrete', 'Wall-to-wall carpet', 'Hardwood', 'Heavy loading', 'Bamboo', 'Slate', 'Laminate', 'Other', 'See Remarks', 'Ceramic/Porcelain', 'Concrete Slab', 'Cork', 'Ceramic', 'Tile', 'Laminate Flooring', 'Mixed', 'Wood', 'Engineered hardwood', 'Carpet over Hardwood', 'Mixed Flooring', 'Stone', 'Linoleum', 'Vinyl', 'Marble', 'Basement Sub-Floor', 'Parquet', 'Basement Slab', 'Vinyl Plank', 'Cork Flooring', 'Wall to Wall Carpet', 'Engineered Wood', 'Subfloor', 'Carpet', 'Ceramic Tile'}


Let's divide into next categories:
- Carpet: Includes all variations of carpet.
- Wood: Includes bamboo, engineered wood, hardwood, etc.
- Tile: Covers ceramic, porcelain, slate, and other types of tiles.
- Vinyl: Includes all vinyl-based flooring types.
- Laminate: Groups laminate and laminate flooring.
- Concrete: Includes concrete and concrete slabs.
- Other: For flooring types that are not easily categorized (like marble, granite, subfloor, etc.).

In [51]:
flooring_categories = {
    'carpet': ['carpet', 'carpet over softwood', 'carpet over hardwood', 'carpeted', 'wall to wall carpet', 
               'wall-to-wall carpet'],
    'wood': ['bamboo', 'engineered wood', 'engineered hardwood', 'hardwood', 'parquet', 'softwood', 'wood'],
    'tile': ['ceramic', 'ceramic tile', 'ceramic/porcelain', 'porcelain tile', 'non-ceramic tile', 'slate', 
             'stone', 'tile'],
    'vinyl': ['cushion/lino/vinyl', 'vinyl', 'vinyl plank'],
    'laminate': ['laminate', 'laminate flooring'],
    'concrete': ['concrete', 'concrete slab'],
    'other': ['basement slab', 'basement sub-floor', 'granite', 'heavy loading', 'linoleum', 'marble', 'mixed', 
              'mixed flooring', 'see remarks', 'subfloor', 'other']
}

In [52]:
df['Flooring new'] = df['Flooring']
df['Flooring new'] = df['Flooring new'].apply(lambda x: check_type(x, flooring_categories))

df['Flooring new'].value_counts()

Flooring new
carpet      12529
wood        10973
laminate     4223
tile         4071
vinyl        2197
other        1009
concrete       86
Name: count, dtype: int64

# Roof

In [53]:
df.Roof.value_counts()

Roof
[]                                          23661
Asphalt Shingles                             3534
["Asphalt Shingle"]                          2916
["Asphalt Shingles"]                         2907
Asphalt Shingle                              2486
                                            ...  
["Vinyl", "Membrane", "Mixed", "Rubber"]        1
Flat, Rubber                                    1
Flat Torch Membrane, Shingle                    1
["Asphalt", "Vinyl", "Wood"]                    1
["Asphalt Shingles", "Tar & Gravel"]            1
Name: count, Length: 247, dtype: int64

In [54]:
unique_roof_features = get_unique_values('Roof')
print(len(unique_roof_features), unique_roof_features)

51 {'', 'Cedar Shakes', 'Shingle', 'Asphalt Shingle', 'Cedar shake', 'Wood Shingles', 'Fiberglass', 'Asphalt/Gravel', 'Flat Torch Membrane', 'Fibreglass Shingle', 'Asphalt Torch On', 'Tar &amp; Gravel', 'Tar/Gravel', 'Steel', 'Pine Shake', 'Asphalt & Gravel', 'Concrete', 'Slate', 'Tar & gravel', 'Fiberglass Shingles', 'Other', 'See Remarks', 'SBS Roofing System', 'Vinyl Shingles', 'Metal', 'Clay Tile', 'Tin', 'Rolled/Hot Mop', 'Metal Shingles', 'Roll Roofing', 'Tile', 'Conventional', 'Mixed', 'Wood', 'Asphalt', 'Tar & Gravel', 'Asphalt shingle', 'Flat', 'Asphalt Shingles', 'Wood Shingle', 'Cedar Shake', 'Vinyl', 'Rubber', 'Pine Shakes', 'Shake', 'None', 'EPDM Membrane', 'Membrane', 'Asphalt Rolled', 'Concrete Tiles', 'Unknown'}


Categories:
- Asphalt: Includes all types of asphalt and asphalt shingles.
- Cedar Shake: Includes cedar shake and shakes.
- Clay: For clay tiles.
- Concrete: Covers concrete and concrete tiles.
- Fiberglass: Includes fiberglass and fiberglass shingles.
- Flat: Includes flat roofing, EPDM, membrane, and flat torch membrane types.
- Metal: Covers metal, metal shingles, steel, and tin.
- Pine Shake: Includes pine shake and shakes.
- Rubber: For rubber roofing.
- SBS: For SBS roofing system.
- Shake: Covers shake roofing.
- Shingle: Includes regular shingles and vinyl shingles.
- Slate: For slate roofing.
- Tar: Includes various types of tar and gravel roofing.
- Tile: For tile roofing.
- Wood: Covers wood shingles and wood shake.
- Other: For unconventional or mixed roofing, or cases where the material is unclear (like 'conventional', 'other', 'mixed').

In [55]:
roofing_categories = {
    'asphalt': ['asphalt', 'asphalt & gravel', 'asphalt rolled', 'asphalt shingle', 'asphalt shingles', 'asphalt torch on', 'asphalt shingle', 'asphalt/gravel'],
    'cedar shake': ['cedar shake', 'cedar shakes'],
    'clay': ['clay tile'],
    'concrete': ['concrete', 'concrete tiles'],
    'fiberglass': ['fiberglass', 'fiberglass shingles', 'fibreglass shingle'],
    'flat': ['flat', 'flat torch membrane', 'membrane', 'epdm membrane'],
    'metal': ['metal', 'metal shingles', 'steel', 'tin'],
    'pine shake': ['pine shake', 'pine shakes'],
    'rubber': ['rubber'],
    'sbs': ['sbs roofing system'],
    'shake': ['shake'],
    'shingle': ['shingle', 'vinyl shingles'],
    'slate': ['slate'],
    'tar': ['tar & gravel', 'tar & gravel', 'tar &amp; gravel', 'tar/gravel'],
    'tile': ['tile'],
    'wood': ['wood', 'wood shingle', 'wood shingles'],
    'other': ['conventional', 'mixed', 'other'],
}

In [56]:
df['Roof new'] = df['Roof']
df['Roof new'] = df['Roof new'].apply(lambda x: check_type(x, roofing_categories))
df['Roof new'].value_counts()

Roof new
asphalt        14182
metal            960
fiberglass       524
other            136
flat             115
cedar shake       98
shingle           83
tile              53
shake             42
clay              40
tar               37
wood              25
rubber            20
concrete          18
pine shake        10
sbs                3
slate              1
Name: count, dtype: int64

# Property Type

In [57]:
df['Property Type'].value_counts()

Property Type
Single Family        51072
Condo                24990
Vacant Land          11268
Condo/Townhome        2914
Townhome              2727
MultiFamily           1821
Duplex                1817
Agricultural           441
Farm                   276
Mobile Home            271
Manufactured Home      213
Timeshare                1
Name: count, dtype: int64

# Waterfront

In [58]:
df.Waterfront.value_counts()

Waterfront
Yes    178
Name: count, dtype: int64

In [59]:
df['Waterfront new'] = df['Waterfront']
df.loc[df['Waterfront new'].isna(), 'Waterfront new'] = 'No'

df['Waterfront new'].value_counts()

Waterfront new
No     97633
Yes      178
Name: count, dtype: int64

# Subdivision

In [60]:
df.Subdivision.value_counts()[:30]

Subdivision
NONE                    1455
Whalley                  718
Willoughby Heights       511
Downtown VW              479
Coquitlam West           374
Brighouse                367
Brentwood Park           351
Metrotown                311
Abbotsford West          299
White Rock               285
Central Abbotsford       283
Fleetwood Tynehead       279
Mission BC               271
Beltline                 270
Langley City             242
Yaletown                 241
West Cambie              210
West End VW              210
Cloverdale BC            202
King George Corridor     201
Abbotsford East          197
Sullivan Station         194
West Newton              194
Grandview Surrey         193
Cambie                   184
Collingwood VE           183
East Newton              182
Marpole                  179
Guildford                165
West Central             157
Name: count, dtype: int64

# Sewer

In [61]:
df.Sewer.value_counts()

Sewer
[]                                                        10549
Sanitary sewer                                             9671
["Municipal sewage system"]                                7379
["Sanitary sewer"]                                         4812
["Septic System"]                                          4682
                                                          ...  
Perc Test On File, Septic Needed                              1
["Perc Test On File", "Septic Needed"]                        1
["Sewer Available", "Sewer Connected", "Sewer To Lot"]        1
Septic System: Common, Sewer To Lot                           1
["Municipal/Community", "Septic Tank &amp; Field"]            1
Name: count, Length: 209, dtype: int64

In [62]:
unique_sewer_features = get_unique_values('Sewer')
print(len(unique_sewer_features), unique_sewer_features)

45 {'', 'Open Discharge', 'Sewer Available', 'Septic tank', 'Sanitary sewer', 'Septic Needed', 'No sewage system', 'Septic Tank &amp; Field', 'Lagoon', 'Septic System: Common', 'Pump', 'Septic Tank and Field', 'Unknown', 'Outhouse', 'Mound Septic', 'See Remarks', 'Other', 'Perc Test On File', 'Aerobic Treatment System', 'Tank &amp; Straight Discharge', 'Liquid Surface Dis', 'Sewer Connected', 'Private sewer', 'Mound', 'Outflow Tank', 'Private Sewer', 'Septic Tank &amp; Mound', 'Engineered Septic', 'Holding Tank', 'Sewer To Lot', 'Septic Tank', 'Municipal/Community', 'Sewer', 'None', 'Public Sewer', 'Storm sewer', 'Septic Field', 'Low Pressure Sewage Sys', 'Facultative Lagoon', 'Attached to Municipal', 'Municipal sewage system', 'Shared Septic', 'Gravity', 'Septic System', 'See remarks'}


In [63]:
sewage_categories = {
    'municipal': ['municipal/community', 'municipal sewage system', 'sanitary sewer', 'sewer', 'sewer connected', 
                  'sewer to lot', 'sewer available', 'public sewer', 'attached to municipal'],
    'septic': ['septic tank', 'septic system', 'septic system: common', 'septic field', 'septic tank and field', 
               'septic tank & mound', 'mound septic', 'septic tank & field', 'septic needed', 'engineered septic'],
    'private': ['private sewer', 'private sewer', 'holding tank', 'low pressure sewage sys', 'shared septic'],
    'alternative': ['aerobic treatment system', 'facultative lagoon', 'lagoon', 'outflow tank', 'open discharge', 
                    'liquid surface dis', 'pump', 'tank & straight discharge'],
    'none': ['no sewage system', 'outhouse', 'none'],
}

In [64]:
df['Sewer new'] = df['Sewer']
df['Sewer new'] = df['Sewer new'].apply(lambda x: check_type(x, sewage_categories))
df['Sewer new'] = df['Sewer new'].fillna('none')

df['Sewer new'].value_counts()

Sewer new
none           60523
municipal      28405
septic          8447
private          273
alternative      163
Name: count, dtype: int64

# Additional Features 

Let's add some additional features like Pool, Gerden, View and Balcony. 

In [65]:
mix_features = ['Basement', 'Exterior', 'Features', 'Fireplace', 'Garage', 'Heating', 'Parking']

In [66]:
df['Combined'] = df[mix_features].astype(str).agg(','.join, axis=1)
df.head(1)

,streetAddress,addressLocality,addressRegion,postalCode,latitude,longitude,description,price,priceCurrency,property-beds,property-baths,property-sqft,Acreage,Air Conditioning,Basement,Bath,Exterior,Features,Fireplace,Garage,Heating,MLS® #,Property Tax,Property Type,Roof,Sewer,Square Footage,Subdivision,Waterfront,Parking,Flooring,Fireplace Features,Square Footage new,Garage new,Parking new,Basement new,Exterior new,Fireplace new,Heating new,Flooring new,Roof new,Waterfront new,Sewer new,Combined
7,"1701 Coursier Avenue Unit# 1301, Revelstoke, BC V0E2S3",Revelstoke,BC,V0E2S3,50.976585,-118.173149,"Don't miss out on this rare 3 bedroom, 2 bath corner unit at Mackenzie Plaza! This fully furnished turn-key short term rental with proven revenue is ready for its new owner. Sit on the patio and enjoy the evening sun of this west facing condo which overlooks Mt Macpherson, Boulder Mountain and Frisbee Ridge. These units can be rented short term, long term or lived in permanently. Included is an underground parking stall and storage locker. The building features elevator access from parking. This amazing location is only minutes to Revelstoke Mountain Resort and downtown Revelstoke, and directly across the road from the new Cabot Revelstoke golf course. Take advantage of the commercial amenities that await on the ground floor which includes a grocery store, co-working space, gym, liquor store, art gallery and more. The resort shuttle bus stops directly across the street so leave your car at home. Book a showing today! (id:38686)",839000.0,CAD,3.0,2.0,891,0.0,NaN,NaN,0.0,NaN,"Underground, Heated Garage",NaN,NaN,Heat Pump,10334677,"$3,478",Condo,NaN,Municipal sewage system,891 SQFT,NaN,NaN,"Underground, Heated Garage","Tile, Carpeted, Vinyl",NaN,891,Yes,Yes,NaN,NaN,No,heat pump,carpet,NaN,No,municipal,"nan,nan,Underground, Heated Garage,nan,nan,Heat Pump,Underground, Heated Garage"


In [67]:
##### Extract unique features
unique_mix_features = set()

for features in df['Combined'].dropna():
    # Remove brackets and extra quotes
    cleaned_features = features.replace("[", "").replace("]", "").replace('"', '').replace("'", "")
    # Split by comma
    feature_list = [f.strip() for f in cleaned_features.split(",")]
    unique_mix_features.update(feature_list)

print(len(unique_mix_features), unique_mix_features)

457 {'', 'Garage; Single', 'Carport & Garage', 'Fireplace(s)', 'Not known', 'Water Feature', 'Rear', 'Raised Hearth', 'Double Carport', 'Carport Quad+', 'Parking Lot', 'Security System', 'Gas Log', 'Conventional', 'Double Garage Attached', 'River View', 'Shared', 'Stall', '2 Car Detached', 'Parking Pad', 'Fencing: Partial', '5', 'Wood siding', 'Gravel', 'RV Parking Avail.', 'Steel', 'Playground', 'Radiant Heat', 'Wall Mounted Heat Pump', 'Level Land', 'Brick Imitation', 'Cedar', 'In Floor', 'Stove', 'Asphalt', 'Partially finished', 'Low Maintenance Landscape', '220 Volt Wiring', '2', 'Landscaped', 'Double Sided', 'Covered Courtyard', 'Direct vent', 'Partially Landscaped', 'Subdividable Lot', 'Lake Access Property', 'Roughed in', 'Kennel', 'In Floor Heating', 'Partial Basement', 'Garage; Double', 'RV Garage', 'Barbecue', 'nan', 'Lake View', 'Ravine View', 'Uncovered Courtyard', 'Vinyl siding', 'View Lake', 'Central heating', 'Brick Veneer', 'Fruit Trees/Shrubs', 'Drive Through', 'Titled

# Pool

Let's add Pool feature

In [68]:
pool_features = ["swimming pool", "public swimming pool"]

In [69]:
df['Pool'] = df['Combined'].apply(lambda x: check_keywords(x, pool_features))
df['Pool'].value_counts()

Pool
No     96992
Yes      819
Name: count, dtype: int64

# Garden

In [70]:
garden_features = ["vegetable garden", "garden", "fruit trees/shrubs", "private yard", 
                   "partially landscaped", "landscaped"]

In [71]:
df['Garden'] = df['Combined'].apply(lambda x: check_keywords(x, garden_features))
df['Garden'].value_counts()

Garden
No     92391
Yes     5420
Name: count, dtype: int64

# View

In [72]:
view_features = ["View Downtown", "River View", "View City", "View Lake", "Lake View", 
                 "Ravine View", "River Valley View"]

view_features = [f.lower() for f in view_features]

In [73]:
def check_view(features, keywords):

    try:
        cleaned_features = features.replace("[", "").replace("]", "").replace('"', '').replace("'", "").replace(";", ",")
        list_to_check = [f.strip().lower() for f in cleaned_features.split(',')]
        common = set(list_to_check) & set(keywords)
        
        if common: 
            return list(common)[0]  
            
    except AttributeError:
        return np.nan
    
    return np.nan

In [74]:
df['View'] = df['Combined'].apply(lambda x: check_view(x, view_features))
df['View'].value_counts()

View
view city            219
river valley view    200
view downtown        100
view lake             83
ravine view           81
lake view             59
river view            40
Name: count, dtype: int64

In [75]:
df.loc[df['View'].isin(['view lake', 'lake view']), 'View'] = 'Lake'
df.loc[df['View'].isin(['view downtown']), 'View'] = 'Downtown'
df.loc[df['View'].isin(['view city']), 'View'] = 'City'
df.loc[df['View'].isin(['river view']), 'View'] = 'River'
df.loc[df['View'].isin(['ravine view', 'river valley view']), 'View'] = 'Valley'

df['View'].value_counts()

View
Valley      281
City        219
Lake        142
Downtown    100
River        40
Name: count, dtype: int64

# Balcony

In [76]:
balcony_features = ['Balcony', 'Balcony/Deck', 'Balcony/Patio']
balcony_features = [f.lower() for f in balcony_features]

In [77]:
df['Balcony'] = df['Combined'].apply(lambda x: check_keywords(x, balcony_features))
df['Balcony'].value_counts()

Balcony
No     94342
Yes     3469
Name: count, dtype: int64

In [78]:
df = df.drop(columns=['streetAddress', 'postalCode', 'description', 'priceCurrency', 'Air Conditioning', 
                 'Basement', 'Exterior', 'Features', 'Fireplace', 'Garage', 'Heating', 'MLS® #', 'Roof', 
                 'Sewer', 'Waterfront', 'Parking', 'Flooring', 'Fireplace Features', 'Combined', 
                 'Subdivision', 'property-sqft', 'Square Footage', 'Bath', 'Property Tax'])

In [79]:
df = df.rename(columns={'addressLocality': 'City', 
                        'addressRegion': 'Province', 
                        'latitude': 'Latitude', 
                        'longitude': 'Longitude', 
                        'price': 'Price',
                        'property-baths': 'Bathrooms', 
                        'property-beds': 'Bedrooms', 
                        'Square Footage new': 'Square Footage', 
                        'Garage new': 'Garage',	
                        'Parking new': 'Parking',	
                        'Basement new': 'Basement', 
                        'Exterior new': 'Exterior', 
                        'Fireplace new': 'Fireplace', 
                        'Heating new': 'Heating', 
                        'Flooring new': 'Flooring', 
                        'Roof new': 'Roof', 
                        'Waterfront new': 'Waterfront', 
                        'Sewer new': 'Sewer'})

In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 97811 entries, 7 to 211
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   City            97811 non-null  object 
 1   Province        97811 non-null  object 
 2   Latitude        97811 non-null  object 
 3   Longitude       97811 non-null  object 
 4   Price           97811 non-null  object 
 5   Bedrooms        84879 non-null  object 
 6   Bathrooms       97041 non-null  object 
 7   Acreage         97811 non-null  float64
 8   Property Type   97811 non-null  object 
 9   Square Footage  46732 non-null  object 
 10  Garage          97811 non-null  object 
 11  Parking         97811 non-null  object 
 12  Basement        37632 non-null  object 
 13  Exterior        46074 non-null  object 
 14  Fireplace       97811 non-null  object 
 15  Heating         75911 non-null  object 
 16  Flooring        35088 non-null  object 
 17  Roof            16347 non-null  object

# Types

In [81]:
df['Square Footage'] = df['Square Footage'].str.replace(',','')

In [82]:
number_columns = ['Latitude', 'Longitude', 'Price', 'Bedrooms', 'Bathrooms', 'Acreage', 
                  'Square Footage']

for col in number_columns: 
    df[col] = df[col].astype(float)

In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 97811 entries, 7 to 211
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   City            97811 non-null  object 
 1   Province        97811 non-null  object 
 2   Latitude        97811 non-null  float64
 3   Longitude       97811 non-null  float64
 4   Price           97811 non-null  float64
 5   Bedrooms        84879 non-null  float64
 6   Bathrooms       97041 non-null  float64
 7   Acreage         97811 non-null  float64
 8   Property Type   97811 non-null  object 
 9   Square Footage  46732 non-null  float64
 10  Garage          97811 non-null  object 
 11  Parking         97811 non-null  object 
 12  Basement        37632 non-null  object 
 13  Exterior        46074 non-null  object 
 14  Fireplace       97811 non-null  object 
 15  Heating         75911 non-null  object 
 16  Flooring        35088 non-null  object 
 17  Roof            16347 non-null  object

# Remove inconsistent data

In [84]:
data = df.copy()

### Square Footage, Bedrooms, and Bathrooms

Let's remove all houses with out Bedroom or Bathrooms

In [85]:
data.dropna(subset=["Bedrooms", "Bathrooms", "Square Footage"], inplace=True)
data[["Bedrooms", "Bathrooms", "Square Footage"]].describe()

,Bedrooms,Bathrooms,Square Footage
count,45188.000000,45188.000000,45188.000000
mean,3.207245,2.513477,1780.138045
std,1.647695,1.428373,1985.084438
min,0.000000,0.000000,0.000000
25%,2.000000,2.000000,969.000000
50%,3.000000,2.000000,1410.000000
75%,4.000000,3.000000,2191.000000
max,38.000000,26.000000,290034.000000


Remove also rows with 'Square Footage' less then 120 sqft.

In [86]:
data = data[data["Square Footage"] > 120]

Remove rows with no bedrooms or bathrooms

In [87]:
# data = data[(data["Bedrooms"] > 0) & (data["Bathrooms"] > 0)]

In [88]:
data[["Bedrooms", "Bathrooms", "Square Footage"]].describe()

,Bedrooms,Bathrooms,Square Footage
count,45086.000000,45086.000000,45086.000000
mean,3.209644,2.516524,1784.155348
std,1.636750,1.424381,1985.528762
min,0.000000,0.000000,140.000000
25%,2.000000,2.000000,972.000000
50%,3.000000,2.000000,1413.000000
75%,4.000000,3.000000,2194.000000
max,37.000000,26.000000,290034.000000


### Province and City

In [89]:
data.Province.value_counts()

Province
BC    23144
SK     5723
AB     5317
ON     3872
NS     2418
NB     2237
NL     1417
PE      799
MB       93
YT       53
NT       13
Name: count, dtype: int64

In [90]:
data.City.unique().shape

(3171,)

In [91]:
data.City.value_counts()[:10]

City
Vancouver    3861
Surrey       3454
Calgary      2176
Richmond     1377
Kelowna      1356
Burnaby      1340
Langley      1191
Saskatoon    1066
Regina        975
Coquitlam     971
Name: count, dtype: int64

We have 3112 different cities in dataset. Perhebs, we will not use this column while modeling

### Price

In [92]:
data.Price.value_counts()

Price
499900.0     385
599900.0     330
699900.0     284
399900.0     279
599000.0     277
            ... 
1858900.0      1
1085952.0      1
496000.0       1
1633000.0      1
1109000.0      1
Name: count, Length: 5538, dtype: int64

We have a lot of houses with unrealistic low price. Perhebs. some position from rent accidantly were put inro sell patr. Let's delete all houses with price less then 50 000.

In [93]:
data = data[data.Price >= 50_000]

### Property Type

In [94]:
data['Property Type'].value_counts()

Property Type
Single Family        26619
Condo                12593
Townhome              2237
Condo/Townhome        1655
Duplex                1056
MultiFamily            281
Mobile Home            228
Manufactured Home      211
Vacant Land             16
Name: count, dtype: int64

### Garage, Parking, 

In [95]:
data[['Garage', 'Parking']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 44896 entries, 7 to 211
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Garage   44896 non-null  object
 1   Parking  44896 non-null  object
dtypes: object(2)
memory usage: 1.0+ MB


In [96]:
data['Garage'].value_counts()

Garage
Yes    31600
No     13296
Name: count, dtype: int64

In [97]:
data['Parking'].value_counts()

Parking
No     26099
Yes    18797
Name: count, dtype: int64

### Fireplace, Pool, Garden, Balcony

In [98]:
data[['Fireplace', 'Pool', 'Garden', 'Balcony', 'Sewer']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 44896 entries, 7 to 211
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Fireplace  44896 non-null  object
 1   Pool       44896 non-null  object
 2   Garden     44896 non-null  object
 3   Balcony    44896 non-null  object
 4   Sewer      44896 non-null  object
dtypes: object(5)
memory usage: 2.1+ MB


### Basement, Exterior, Fireplace, Heating, Flooring, Roof, Sewer, View

In [99]:
data[['Basement', 'Exterior', 'Heating', 'Flooring', 'Roof', 'View']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 44896 entries, 7 to 211
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Basement  14962 non-null  object
 1   Exterior  17439 non-null  object
 2   Heating   38242 non-null  object
 3   Flooring  15216 non-null  object
 4   Roof      10052 non-null  object
 5   View      12 non-null     object
dtypes: object(6)
memory usage: 2.4+ MB


In [100]:
data = data.drop(columns=['View'])
data.head()

,City,Province,Latitude,Longitude,Price,Bedrooms,Bathrooms,Acreage,Property Type,Square Footage,Garage,Parking,Basement,Exterior,Fireplace,Heating,Flooring,Roof,Waterfront,Sewer,Pool,Garden,Balcony
7,Revelstoke,BC,50.976585,-118.173149,839000.0,3.0,2.0,0.00,Condo,891.0,Yes,Yes,NaN,NaN,No,heat pump,carpet,NaN,No,municipal,No,No,No
8,Boswell,BC,49.471870,-116.770195,1150000.0,3.0,2.0,0.32,Single Family,1881.0,Yes,Yes,NaN,NaN,No,heat pump,NaN,NaN,No,septic,No,No,No
9,West Kelowna,BC,49.825230,-119.603253,149000.0,2.0,1.0,0.00,Single Family,912.0,No,No,NaN,Metal,No,NaN,laminate,tar,No,municipal,No,No,No
10,Kelowna,BC,49.821860,-119.480143,1298000.0,5.0,4.0,0.69,Single Family,4374.0,Yes,No,NaN,NaN,Yes,forced air,NaN,NaN,No,municipal,No,No,No
12,Maple Ridge,BC,49.221673,-122.596637,759900.0,3.0,2.0,0.00,Condo,1254.0,Yes,Yes,NaN,NaN,No,radiant,NaN,NaN,No,none,No,No,No


# Save data

In [101]:
df.to_csv('cleaned_canada.csv', index=False)